<a href="https://colab.research.google.com/github/ovokojo/fintequity/blob/main/SEC_API_Tutorials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install SEC API & load API Key from file (Get a free API key at https://sec-api.io/)
!pip install python-dotenv
!pip install -q sec-api

from dotenv import dotenv_values
from google.colab import files

# Upload the .env file from your local machine
uploaded = files.upload()

# Get the file name of the uploaded file
filename = next(iter(uploaded))

# Load the environment variables from the uploaded file
env_vars = dotenv_values(filename)

# Extract the API key from the environment variables
API_KEY = env_vars.get("API_KEY")

print("API key loaded successfully")

Saving .ENV to .ENV
API key loaded successfully


In [3]:
import requests

def getLatest10K(ticker: str):
    """
    Retrieves the most recent Form 10-K filing for a given stock ticker symbol.

    The function sends a POST request to the SEC API, querying for the latest Form 10-K
    document filed by the company associated with the provided ticker symbol.

    Parameters:
    - ticker (str): The stock ticker symbol of the company for which the Form 10-K is requested.

    Returns:
    - dict: A dictionary containing the JSON response with the latest Form 10-K filing
            information if the request is successful (HTTP status code 200).
    - int: The HTTP status code of the response if the request is unsuccessful.

    Note: This function requires an 'API_KEY' variable to be set with a valid SEC API key.
          The API key should be obtained and set in the environment or directly in the code
          before calling this function.
    """

    # Define the endpoint URL for the SEC API. Replace 'API_KEY' with your actual SEC API key.
    endpoint = f"https://api.sec-api.io?token={API_KEY}"

    # Send a POST request to the SEC API with the query for the latest 10-K form.
    response = requests.post(endpoint, json={
        "query": {
            "query_string": {
                "query":  f"ticker:{ticker} AND formType:\"10-K\""
            }
        },
        "from": "0",
        "size": "1",
        "sort": [{ "filedAt": { "order": "desc" }}]
    })

    # Check if the request was successful (HTTP status code 200).
    if response.status_code == 200:
        # Print and return the JSON response containing the 10-K filing information.
        print(response.json())
        return response.json()
    else:
        # If the request was unsuccessful, return the HTTP status code.
        return response.status_code

# Example usage: Get the latest Form 10-K filing for American Express Company (ticker: AXP).
latest10K = getLatest10K('AXP')


{'total': {'value': 32, 'relation': 'eq'}, 'query': {'from': 0, 'size': 1}, 'filings': [{'id': 'b1c96c31126cc9e5196ecc3a5c8b0bb6', 'accessionNo': '0000004962-24-000013', 'cik': '4962', 'ticker': 'AXP', 'companyName': 'AMERICAN EXPRESS CO', 'companyNameLong': 'AMERICAN EXPRESS CO (Filer)', 'formType': '10-K', 'description': 'Form 10-K - Annual report [Section 13 and 15(d), not S-K Item 405]', 'filedAt': '2024-02-09T14:08:08-05:00', 'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/4962/000000496224000013/0000004962-24-000013.txt', 'linkToHtml': 'https://www.sec.gov/Archives/edgar/data/4962/000000496224000013/0000004962-24-000013-index.htm', 'linkToXbrl': '', 'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/4962/000000496224000013/axp-20231231.htm', 'entities': [{'companyName': 'AMERICAN EXPRESS CO (Filer)', 'cik': '4962', 'irsNo': '134922250', 'stateOfIncorporation': 'NY', 'fiscalYearEnd': '1231', 'type': '10-K', 'act': '34', 'fileNo': '001-07657', 'filmNo': '24613930

In [ ]:
## Income Statement Extraction Example

from sec_api import XbrlApi
import pandas as pd

xbrlApi = XbrlApi(API_KEY)

url_10k = latest10K['filings'][0]['linkToFilingDetails']

xbrl_json = xbrlApi.xbrl_to_json(htm_url=url_10k)


# convert XBRL-JSON of income statement to pandas dataframe
def get_income_statement(xbrl_json):
    income_statement_store = {}

    # iterate over each US GAAP item in the income statement
    for usGaapItem in xbrl_json['StatementsOfIncome']:
        values = []
        indicies = []

        for fact in xbrl_json['StatementsOfIncome'][usGaapItem]:
            # only consider items without segment. not required for our analysis.
            if 'segment' not in fact:
                index = fact['period']['startDate'] + '-' + fact['period']['endDate']
                # ensure no index duplicates are created
                if index not in indicies:
                    values.append(fact['value'])
                    indicies.append(index)

        income_statement_store[usGaapItem] = pd.Series(values, index=indicies)

    income_statement = pd.DataFrame(income_statement_store)
    # switch columns and rows so that US GAAP items are rows and each column header represents a date range
    return income_statement.T

income_statement = get_income_statement(xbrl_json)


print("Income statement from AMEX's 2022 10-K filing as dataframe")
print('------------------------------------------------------------')
income_statement

Income statement from AMEX's 2022 10-K filing as dataframe
------------------------------------------------------------


,2023-01-01-2023-12-31,2022-01-01-2022-12-31,2021-01-01-2021-12-31
NoninterestIncome,47381000000,42967000000,34630000000
InterestAndFeeIncomeLoansAndLeases,17697000000,11967000000,8850000000
InterestAndDividendIncomeSecurities,128000000,96000000,83000000
InterestIncomeDepositsWithFinancialInstitutions,2158000000,595000000,100000000
InterestAndDividendIncomeOperating,19983000000,12658000000,9033000000
InterestExpenseDeposits,4865000000,1527000000,458000000
InterestExpenseLongTermDebt,1984000000,1236000000,825000000
InterestExpense,6849000000,2763000000,1283000000
InterestIncomeExpenseNet,13134000000,9895000000,7750000000
RevenuesNetOfInterestExpense,60515000000,52862000000,42380000000
